In [46]:
import json
import warnings
warnings.filterwarnings("ignore")
with open("intent.json",'r') as file:
    data1=json.load(file)
with open("intent.json",'r') as file:
    data2=json.load(file)

data=data1
data['intents'].extend(data2['intents'])
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'Is anyone there?',
    'Hi there',
    'Hello',
    'Hey there',
    'Howdy',
    'Hola',
    'Bonjour',
    'Konnichiwa',
    'Guten tag',
    'Ola'],
   'responses': ['Hello there. Tell me how are you feeling today?',
    'Hi there. What brings you here today?',
    'Hi there. How are you feeling today?',
    'Great to see you. How do you feel currently?',
    "Hello there. Glad to see you're back. What's going on in your world right now?"]},
  {'tag': 'morning',
   'patterns': ['Good morning'],
   'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]},
  {'tag': 'afternoon',
   'patterns': ['Good afternoon'],
   'responses': ['Good afternoon. How is your day going?']},
  {'tag': 'evening',
   'patterns': ['Good evening'],
   'responses': ['Good evening. How has your day been?']},
  {'tag': 'night',
   'patterns': ['Good night'],
   'responses': ['Good night. Get some prop

In [47]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
labels = []
tags=[]
all_words=[]
Xy= []
lemma=WordNetLemmatizer()
for intent in data['intents']:
    tag= intent['tag']
    if tag not in labels:
        tags.append(tag)  # Keep unique labels
        for pattern in intent['patterns']:
            pattern=pattern.lower()
            pattern=re.sub(r"[^A-Za-z0-9]"," ",pattern)
            all_words.extend(word_tokenize(pattern))
            Xy.append((pattern,tag))

all_words=[lemma.lemmatize(word) for word in all_words if word not in set(stopwords.words('english'))]
all_words=sorted(set(all_words))

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
def bagofwords(tokens, all_words):
    vector = np.zeros(len(all_words), dtype=np.float32)
    for idx, word in enumerate(all_words):
        if word in tokens:
            vector[idx] = 1.0
    return vector


In [49]:
xtrain=[]
ytrain=[]
for (pattern,tag) in Xy:
    label=tags.index(tag)
    ytrain.append(label)
    xtrain.append(bagofwords(pattern,all_words))

In [50]:
from tensorflow.keras.utils import to_categorical
xtrain = np.array(xtrain, dtype=np.float32)
ytrain = np.array(ytrain, dtype=np.float32)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
model_dl=Sequential()
model_dl.add(Dense(128,activation='relu',input_shape=(xtrain.shape[1],)))
model_dl.add(Dropout(0.2))
model_dl.add(Dense(64,activation='relu'))
model_dl.add(Dense(len(tags),activation='softmax'))
model_dl.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_dl.fit(xtrain,ytrain,epochs=100,callbacks=[callback])

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0237 - loss: 5.0451    
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1336 - loss: 4.9221 
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1659 - loss: 4.7011 
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1099 - loss: 4.3509 
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1056 - loss: 3.9732 
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1272 - loss: 3.7211 
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2198 - loss: 3.5089 
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2716 - loss: 3.2994 
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3448 - loss: 3.0853 
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3944 - loss: 2.8627 
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4116 - loss: 2.6277 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/st

In [52]:
def process_user_query(user_query):
    tokens = word_tokenize(user_query)
    tokens = [lemma.lemmatize(word.lower()) for word in tokens
              if word.isalpha() and word not in stopwords.words('english')]
    bow = bagofwords(tokens, all_words)  # pass tokens list, not joined string
    return bow.reshape(1, -1)


In [53]:
import random
while True:
    user_query=input("Type: ")
    print(f"User: {user_query}")
    if user_query=='exit':
        print(f"Bot:See You!")
        break
    user_query=process_user_query(user_query)
    predicted_prob=model_dl.predict(user_query)[0]
    predicted_label=np.argmax(predicted_prob)
    predicted_tag=tags[predicted_label]
    for intent in data['intents']:
        if intent['tag'] == predicted_tag:
            response = random.choice(intent['responses'])
            break
    print(f"Bot: {response}\n")
    


User: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Bot: Great to see you. How do you feel currently?

User: Who are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: I'm Pandora, a Therapeutic AI Assitant designed to assist you. Tell me about yourself.

User: what will you do
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: I'm Pandora!

User: can you help me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Bot: Yes, sure. How can I help you?

User: i am feeling depressed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: Sometimes when we are depressed, it is hard to care about anything. It can be hard to do the simplest of things. Give yourself time to heal.

User: i am feeling lonely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: Why do you think you feel this way?

User: because i have no person to care me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Bot: You can call me Pandora.

User: can you understand
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Bot: It sound like i'm not being very helpful right now.

User: good night
1/1 ━━━

In [54]:
model_dl.save('model_dl.h5')

In [ ]:
import pickle as pkl

with open("tags.pkl", "wb") as f:
    pkl.dump(tags, f)


with open("all_words.pkl", "wb") as f:
    pkl.dump(all_words, f)
